In [1]:
import os
os.chdir("../")

In [2]:
import os
import urllib.request as request
import zipfile
from src.datascience import logger
from dataclasses import dataclass

In [3]:
from src.datascience.utils.common import read_yaml, create_directories
from src.datascience.constants import *

In [4]:

from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_URL:str
    local_data_file:Path
    unzip_dir:Path

In [5]:
class ConfigurationManager:
    
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH,
        schema_filepath=SCHEMA_FILE_PATH
    ):
        # ensure inputs are Path objects (read_yaml requires Path)
        config_filepath = Path(config_filepath)
        params_filepath = Path(params_filepath)
        schema_filepath = Path(schema_filepath)

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schemas = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=Path(config.root_dir),
            source_URL=config.source_url,
            local_data_file=Path(config.local_data_file),
            unzip_dir=Path(config.unzip_dir)
        )
        return data_ingestion_config


In [6]:
class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config = config
### Downloading the zip file
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename,headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )

            logger.info(f"{filename} downloaded with following info: \n{headers}")
        else:
            logger.info(f"File already exists of path: {self.config.local_data_file}")


### Extracting the zip file
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)
        logger.info(f"File extracted at : {unzip_path}")


In [7]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.data_ingestion_config()

    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()

except Exception as e:
    raise e

[2025-12-11 00:56:43,231 : INFO  : common : yaml file: config/config.yaml loaded successfully]
[2025-12-11 00:56:43,233 : INFO  : common : yaml file: params.yaml loaded successfully]
[2025-12-11 00:56:43,234 : INFO  : common : yaml file: schema.yaml loaded successfully]
[2025-12-11 00:56:43,236 : INFO  : common : created directory at: artifacts]
[2025-12-11 00:56:43,237 : INFO  : common : created directory at: artifacts/data_ingestion]
[2025-12-11 00:56:44,355 : INFO  : 2856810451 : artifacts/data_ingestion/data.zip downloaded with following info: 
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 8D97:A0972:EE7BD:1BB263:6939C970
Accept-Rang

In [8]:
import os

for root, dirs, files in os.walk(".", topdown=True):
    if "config.yaml" in files:
        print(os.path.join(root, "config.yaml"))


./config/config.yaml
